In [ ]:
!pip install scanpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install leidenalg

In [ ]:
%pip install episcanpy

In [ ]:
import scanpy as sc
from scipy import stats
import numpy as np
import pandas as pd
import episcanpy as epi
from scipy.sparse import issparse
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn import metrics
import math
from scipy import stats

In [ ]:
def preprocessing_rna(
    adata,
    min_genes=200, 
    min_cells=0.01, 
    target_sum=1e4
):
    """
    preprocessing
    """
    print('Raw dataset for scRNA shape: {}'.format(adata.shape))
    if not issparse(adata.X):
        adata.X = scipy.sparse.csr_matrix(adata.X)

    #Filtering cells
    #sc.pp.filter_cells(adata, min_genes=min_genes)

    #Flitering genes
    sc.pp.filter_genes(adata, min_cells=min_cells)

    #LogNormalize
    sc.pp.normalize_total(adata, target_sum=target_sum)
    sc.pp.log1p(adata)
    adata.raw = adata

    print('Processed dataset shape: {}'.format(adata.shape))
    return adata

In [ ]:
raw_rna = sc.read_h5ad("drive/MyDrive/B2_Model/dataset/E18_mouse_brain/e18_mouse_brain_fresh_5k_rna_fragments.h5ad")
trans_rna_test = np.load("drive/MyDrive/output_0801_02/rna_trans_output_test.npy")
trans_rna_train = np.load("drive/MyDrive/output_0801_02/rna_trans_output_train.npy")
target_name_test = pd.read_table("drive/MyDrive/output_0801_02/rna_targets_test.txt", header=None)
target_name_train = pd.read_table("drive/MyDrive/output_0801_02/rna_targets_train.txt", header=None)

In [ ]:
def merge_train_test(raw_rna, trans_rna_test, trans_rna_train, target_name_test, target_name_train):
  pre_rna = preprocessing_rna(raw_rna)

  pre_rna = pre_rna.to_df()
  pre_rna[pre_rna < 0] = 0
  trans_rna_train[trans_rna_train < 0] = 0
  trans_rna_test[trans_rna_test < 0] = 0
  
  target_name = target_name_train.append(target_name_test)
  
  df = pd.DataFrame(pre_rna.loc[target_name.iloc[0][0]]).T
  for i in range(1,target_name.shape[0]):
    cell = target_name.iloc[i][0]
    X = pre_rna.loc[cell]
    X_df = pd.DataFrame(X).T
    df = df.append(X_df)
  
  x = df
  y = pd.DataFrame(trans_rna_train).append(pd.DataFrame(trans_rna_test))
  return x, y

In [ ]:
pre_rna, trans_rna = merge_train_test(raw_rna, trans_rna_test, trans_rna_train, target_name_test, target_name_train)

Raw dataset for scRNA shape: (4482, 32285)
Processed dataset shape: (4482, 24528)


In [126]:
pre_rna

,Xkr4,Gm1992,Gm19938,Gm37381,Rp1,Sox17,Gm37587,Gm37323,Mrpl15,Lypla1,Tcea1,Rgs20,Gm16041,Atp6v1h,Oprk1,Rb1cc1,4732440D04Rik,Alkal1,St18,Pcmtd1,Gm26901,Sntg1,Rrs1,Adhfe1,2610203C22Rik,Vxn,Mybl1,Vcpip1,1700034P13Rik,Sgk3,Mcmdc2,Snhg6,Tcf24,Ppp1r42,Gm15818,Cops5,Cspp1,Arfgef1,Cpa6,Prex2,...,Gm29866,Gm31571,Gm20896,Gm21865,Gm28887,Gm21518,Gm20937,Gm29566,Gm21860,Gm47283,mt-Nd1,mt-Nd2,mt-Co1,mt-Co2,mt-Atp8,mt-Atp6,mt-Co3,mt-Nd3,mt-Nd4l,mt-Nd4,mt-Nd5,mt-Nd6,mt-Cytb,AC125149.3,AC133103.1,AC168977.1,CAAA01118383.1,AC132444.1,AC132444.5,Csprs,AC132444.6,Vamp7,CAAA01165726.1,Spry3,Tmlhe,4933409K07Rik,Gm10931,CAAA01147332.1,AC163611.2,AC149090.1
CGAATCTAGCGATAAG-1,1.849068,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.727925,0.0,0.727925,0.000000,0.0,1.144719,0.0,0.000000,0.0,0.0,0.0,0.727925,0.00000,1.849068,0.0,0.000000,0.0,0.0,0.000000,1.144719,0.0,0.000000,0.0,0.727925,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.139531,2.139531,0.727925,2.836835,0.727925,3.323937,3.361772,0.727925,0.727925,2.702711,1.849068,0.000000,2.836835,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.849068
ACTAACCAGCCACAAT-1,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.681504,0.0,0.681504,0.000000,0.0,1.772278,0.0,0.681504,0.0,0.0,0.0,1.772278,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.681504,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.083058,1.590743,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.714810,2.811183,2.868264,3.908867,1.368778,4.267361,4.280962,2.176431,1.083058,2.922262,1.590743,0.000000,3.614786,0.0,0.0,0.0,0.681504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.083058
AAGCGGGTCATGCTAA-1,2.767714,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.141589,0.000000,0.0,0.000000,0.0,1.141589,0.0,0.0,0.0,0.000000,0.00000,3.004973,0.0,0.000000,0.0,0.0,0.000000,1.141589,0.0,2.000835,0.0,0.000000,0.0,0.0,0.0,1.141589,1.141589,1.141589,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.004973,3.428955,2.893364,4.173652,1.660793,3.994405,4.380456,2.623976,1.660793,3.004973,1.660793,0.000000,3.280193,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.893364
AGGTTAGAGTTACCGG-1,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.469679,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,1.469679,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.469679,2.401844,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.717809,2.040659,2.401844,3.324414,0.000000,3.868484,4.312844,2.040659,0.000000,2.040659,0.000000,0.000000,3.048659,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.040659
GGTACAAAGTAACCCG-1,1.699694,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.699694,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,1.17438,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.174380,1.174380,1.174380,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.297024,2.812587,1.699694,3.242495,0.000000,3.772452,3.663929,2.297024,2.042312,2.812587,1.174380,0.000000,3.050468,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.812587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGTTAAAGCCGCAGT-1,2.673823,1.074284,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.580172,0.0,1.074284,0.0,1.580172,0.0,0.0,0.0,0.000000,0.00000,1.580172,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.580172,2.531105,1.580172,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.531105,2.164654,1.074284,2.164654,0.000000,2.798693,2.531105,0.000000,1.074284,2.164654,0.

In [125]:
trans_rna

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,24488,24489,24490,24491,24492,24493,24494,24495,24496,24497,24498,24499,24500,24501,24502,24503,24504,24505,24506,24507,24508,24509,24510,24511,24512,24513,24514,24515,24516,24517,24518,24519,24520,24521,24522,24523,24524,24525,24526,24527
0,1.663147,0.109256,0.224218,0.000000,0.000000,0.029407,0.000000,0.000000,0.031734,0.093250,0.402017,0.186840,0.067447,0.456139,0.004397,0.835954,0.091459,0.019498,0.000000,0.524770,0.169011,1.251400,0.000000,0.034199,0.007904,0.068370,0.000000,0.224665,0.014350,0.087384,0.021124,0.328695,0.011978,0.018126,0.000000,0.157350,0.952543,0.707718,0.000000,0.028032,...,0.000000,0.018204,0.021861,0.006840,0.000000,0.013940,0.023935,0.000000,0.000000,0.354303,2.790650,2.645078,2.165436,3.547890,0.522097,3.696757,3.877124,1.660267,0.736955,2.734866,1.011465,0.039711,3.237714,0.000000,0.003109,0.000000,0.240655,0.000000,0.000000,0.000000,0.008738,0.179616,0.000000,0.002721,0.065618,0.000000,0.000000,0.000000,0.000000,2.483842
1,0.000000,0.000945,0.000000,0.000000,0.006306,0.006516,0.010368,0.000000,0.300616,0.223780,0.538404,0.051649,0.000000,0.465530,0.008398,0.802421,0.043286,0.024294,0.019348,0.671885,0.000000,0.178112,0.050214,0.000000,0.007218,0.000567,0.065236,0.330960,0.000000,0.003507,0.051654,0.534883,0.003266,0.001692,0.000000,0.418461,0.983463,0.717472,0.000000,0.005328,...,0.000000,0.002780,0.000000,0.000000,0.000000,0.002853,0.014912,0.023447,0.023523,0.337508,3.612530,3.180762,3.007196,4.155331,0.588300,4.324014,4.559604,2.333003,1.266650,3.241597,1.587194,0.163417,3.751316,0.000000,0.004817,0.000000,0.217488,0.017304,0.000000,0.004467,0.000000,0.248866,0.000000,0.000000,0.198377,0.008161,0.000000,0.019283,0.001088,1.267275
2,2.219059,0.138909,0.380466,0.000000,0.018341,0.000000,0.022082,0.015920,0.042016,0.076146,0.508206,0.060252,0.000000,0.310884,0.000000,1.064891,0.048449,0.000335,0.000000,0.903402,0.040852,1.084102,0.065587,0.004737,0.038686,0.000000,0.005464,0.258582,0.020425,0.155049,0.041759,0.054986,0.022370,0.005974,0.063250,0.404599,1.024224,0.667576,0.018989,0.103358,...,0.002539,0.000000,0.024761,0.018552,0.000000,0.015449,0.000000,0.030321,0.001684,0.410180,2.642483,2.094962,1.996636,3.182024,0.410719,3.329800,3.505541,1.769329,0.541616,2.436993,0.837535,0.099688,2.583028,0.001123,0.000000,0.000000,0.164086,0.031876,0.025871,0.000000,0.000000,0.063687,0.011300,0.023190,0.071040,0.061562,0.014024,0.068127,0.014995,2.451772
3,0.155402,0.015174,0.000000,0.000000,0.021751,0.005968,0.000000,0.023821,0.263739,0.296557,0.704553,1.382125,0.015082,0.333106,0.000000,0.856375,0.100981,0.005165,0.006398,0.520054,0.052312,0.044170,0.048523,0.054284,0.002287,0.000000,0.109953,0.248679,0.000000,0.165783,0.041564,0.388536,0.008073,0.025078,0.003303,0.596054,0.999525,0.730850,0.018249,0.048690,...,0.000000,0.011952,0.000000,0.000000,0.008346,0.011042,0.015574,0.000000,0.000769,0.652840,3.098688,2.601895,2.448379,3.581042,0.372364,3.705975,3.840054,1.630992,0.460787,2.465210,1.142532,0.029705,3.059148,0.018052,0.000000,0.000000,0.236931,0.022556,0.000000,0.031035,0.000000,0.136442,0.008916,0.000000,0.286550,0.005937,0.000000,0.006486,0.000362,1.048490
4,0.965651,0.023693,0.038412,0.000000,0.006706,0.000000,0.000000,0.000000,0.217851,0.026635,0.492060,0.082771,0.000000,0.406302,0.012478,0.692195,0.037102,0.022610,0.610386,0.573369,0.087208,1.852780,0.081263,0.035383,0.020346,0.000000,0.000000,0.354165,0.008999,0.000000,0.059114,0.294779,0.000000,0.000000,0.000000,0.430206,0.716268,0.633125,0.000000,0.000000,...,0.000000,0.003370,0.000000,0.000000,0.000000,0.000000,0.005825,0.000000,0.002459,0.106285,3.468284,3.265889,2.678108,4.170661,0.650640,4.193113,4.605741,2.441626,1.087569,3.070115,1.260717,0.105961,3.503724,0.005014,0.000000,0.011168,0.190094,0.016081,0.002355,0.000000,0.000000,0.072424,0.000000,0.000000,0.003637,0.000000,0.000000,0.044237,0.015486,1.883351
...,...,...,...

In [128]:
def ttest_each_gene(pre_rna, trans_rna):
  pvalue_list = list()
  for i in range(0,pre_rna.shape[1]):
    x = pre_rna.iloc[:,i].values
    y = trans_rna.iloc[:,i].values
    Pvalue = stats.ttest_rel(x,y)[1]
    pvalue_list.append(Pvalue)
  pvalue_list = np.array(pvalue_list)
  print("Pvalue >= 0.05:", np.sum(pvalue_list>=0.05))
  print("Pvalue < 0.05:", np.sum(pvalue_list<0.05))

  good_index = np.where(pvalue_list>=0.05)[0]
  bad_index = np.where(pvalue_list<0.05)[0]
  
  good_index_pvalue = {}
  for i in good_index:
    good_index_pvalue[i] = pvalue_list[i]          
  good_index_pvalue = sorted(good_index_pvalue.items(), key = lambda kv:(kv[1], kv[0]))[0:500]
  
  good_gene_name = list()
  for i in range(len(good_index_pvalue)):
    good_gene_name.append(pre_rna.iloc[:,good_index_pvalue[i][0]].name)

  bad_index_pvalue = {}
  for i in bad_index:
    bad_index_pvalue[i] = pvalue_list[i]
  bad_index_pvalue = sorted(bad_index_pvalue.items(), key = lambda kv:(kv[1], kv[0]))[0:500]

  bad_gene_name = list()
  for i in range(len(bad_index_pvalue)):
    bad_gene_name.append(pre_rna.iloc[:,bad_index_pvalue[i][0]].name)

  return good_gene_name, bad_gene_name

In [129]:
good_gene_name, bad_gene_name = ttest_each_gene(pre_rna, trans_rna)

Pvalue >= 0.05: 11053
Pvalue < 0.05: 13475


In [133]:
good_gene_name = pd.DataFrame(data=good_gene_name, columns=['good_gene'])

In [134]:
bad_gene_name = pd.DataFrame(data=bad_gene_name, columns=['bad_gene'])

In [137]:
good_gene_name.to_csv("good_gene.csv")
bad_gene_name.to_csv("bad_gene.csv")